<a href="https://colab.research.google.com/github/rithikkulkarni/Twitter-Sentiment-Stock-Price-Prediction/blob/main/finbert_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/rithikkulkarni/ALDA-Course-Project/refs/heads/main/datasets/stock_tweets.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null  object
 3   Company Name  80793 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert", device="cuda")

In [ ]:
### Function for using FinBERT to label our data for supervised learning
def label_data(tweet_text):
  temp = pipe(tweet_text)
  if 'positive' in str(temp[0]):
    return 'positive'
  elif 'negative' in str(temp[0]):
    return 'negative'
  else:
    return 'neutral'

### Function for using FinBERT that should be faster because it process multiple tweets in parallel
def label_data_batch(tweet_texts, batch_size=32):
    results = pipe(tweet_texts, batch_size=batch_size)
    labels = []
    for res in results:
        if 'positive' in str(res):
            labels.append('positive')
        elif 'negative' in str(res):
            labels.append('negative')
        else:
            labels.append('neutral')
    return labels

In [ ]:
# import numpy as np

# batch_size = 32  # Adjust based on memory and speed
# tweet_batches = np.array_split(df['Tweet'].to_numpy(), len(df) // batch_size + 1)
# label_batches = [label_data_batch(batch.tolist()) for batch in tweet_batches]

# # Flatten the list of label batches
# df['label'] = np.concatenate(label_batches)

import torch
from datasets import Dataset

# Ensure GPU is enabled
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("text-classification", model="ProsusAI/finbert", device=device)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['Tweet']])

# Define a function to process tweets
def label_data_batch(batch):
    results = pipe(batch['Tweet'], batch_size=32)
    return {'label': [res['label'].lower() for res in results]}

# Apply the function to the dataset
labeled_dataset = dataset.map(label_data_batch, batched=True)

# Convert back to DataFrame
df['label'] = labeled_dataset['label']

In [ ]:
df.info()

In [ ]:
df.rename(columns={'label': 'Label'}, inplace=True)
df.info()

In [ ]:
df.Label.value_counts()

In [ ]:
### To be used for stock price

import yfinance as yf

ticker = yf.Ticker("AAPL")
hist = ticker.history(start="2024-02-05", end="2024-02-12")
print(hist)

In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

# Compute the continuous compound sentiment score for each tweet
df['sentiment'] = df['Tweet'].apply(lambda tweet: analyzer.polarity_scores(tweet)['compound'])

print(df.head())


In [ ]:
df.head(50)

In [ ]:
df.to_csv('continuous_labeled_tweets.csv', index=False)